In [ ]:
import sys
sys.path.append('..')

In [1]:
from parser import public_construct, public_match, public_match_exact

In [2]:
from utils import SocketReader, ReaderWrapper

In [3]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDFExpand, HKDF
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.asymmetric import ec

ModuleNotFoundError: No module named 'cryptography'

In [4]:
try:
    sock.close()
except:
    pass

sock = SocketReader("www.google.com", 443, 'udp')

sock.connect()

In [9]:
s = """
8@pla_length
-8@asd
-l@pla()
-8@ddd
"""
public_construct(s, public_match(s, b'\x10\x33\xf4\x44\x65', print=print)[0])

40 0001000000110011111101000100010001100101
['8@pla_length', '8@asd', 'l@pla!0', '8@ddd']
8
8
l!0
111101000100010001100101 24 16
8


ValueError: invalid literal for int() with base 10: 'l'

In [6]:
%debug

> /home/mie/notebooks/parser.py(137)match()
    135                     spec_len = c.split('!')[0]
    136                     if c[0] == 'l':
--> 137                         cline=cline[:ms[name+'_length']]
    138                     elif c[0] != '~':
    139                         mod = 0

ipdb> ms
{'pla_length': 0x16, 'asd': "3"}
ipdb> ms
{'pla_length': 0x16, 'asd': "3"}
ipdb> ms['pla_length']
0x16
ipdb> type(ms['pla_length'])
<class 'parser.Repr'>
ipdb> dir(ms['pla_length'])
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'bytes', 'hex', 'int', 'r', 'str', 'strict_length', 'val', 'value']
ipdb> ms['pla_length'].int
<bound method Repr.int of 0x16>
ipdb> ms['pla_length'].int()
22

In [ ]:
initial_packet_spec = """
1==1?(
    1@fixed_bit
    -2==0(
        2@reserved
        -2@packet_number_length
        -32@version
        -8@destination_connection_id()
        -8@source_connection_id()
    )
)
:1==0?(
    1@fixed_bit=1
    -2long_packet_type
    -4@type_specific_bits
    -32@version
    -8@destination_connection_id()
    -8@source_connection_id()
)
"""

In [112]:
client_hello_spec = """
8==1?24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suites(...16)
    -8@compression()
    -16@extensions(
        ...(
            16==0?16@ext_server_name(
                16@data(
                    8@name_type
                    -16@host_name()
                )
            )
            :16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==10?16@ext_supported_curves(
                16@data(...16)
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==13?16@ext_signature_algorithms(
                16@data(...16)
            )
            :16==50?16@ext_signature_algorithms_cert(
                16@data(...16)
            )
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==16?16@ext_alpn(
                16@alpn_protocols(...8())
            )
            :16==43?16@ext_suppoerted_versions(
                8@supported_versions(...16)
            )
            :16==44?16@ext_cookie(
                16@cookie()
            )
            :16==51?16@ext_key_share(
                16@key_shares(
                    ...(
                        16@group
                        -16@data()
                    )
                )
            )
            :16==18?16@ext_sct(
                16@scts(
                    ...16@data()
                )
            )
            :16==24?16@ext_token_binding(

            )
            :16==42?16@empty()
            :16==45?16@ext_psk_modes(
                8@psk_modes()
            )
            :16==41?16@ext_pre_shared_key(
                16@psk_identities(
                    ...(
                        16@label()
                        -32@obfuscated_ticket_age
                    )
                )
                -16@psk_binders(
                    ...8()
                )
            )
            :16@selector-16@raw_extension()
        )
    )
)"""

client_hello_packet_spec = """
8==22?16@tls_version-16@tls_handshake(%s)
"""%client_hello_spec

In [113]:
private_key = X25519PrivateKey.generate()

public_key = private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)

In [114]:
transcript = hashes.Hash(hashes.SHA256())

In [115]:
client_hello = {
    'client_hello': {
        'version': 0x0303,
        'random': 0xab6618b2bf3898950c1f4820897760317c348c3b9c8288a05e436d4ba4631d9d,
        'session_id': 0x130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6,
        'suites': [0x1301],
        'compression': 0x00,
        'extensions': [
            {
                'ext_server_name': {
                    'data': {
                        'name_type': 0x00, 'host_name': "www.google.com"
                    }
                }, '_selector': 0x0000
            },
            {
                'ext_status_request': {
                    'status_type': 0x01, 'responder_id_list': 0x0000, 'request_extensions': 0x0000
                },
                '_selector': 0x0005
            },
            {
                'ext_supported_curves': {
                    'data': [0x001d]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': 0x00
                }, '_selector': 0x000b
            },
            {
                'ext_signature_algorithms': {
                    'data': [0x0403]
                }, '_selector': 0x000d
            },
            {
                'ext_renegotiation_info': {
                    'secure_renegotitation': ''
                }, '_selector': 0xff01
            },
            {
                'ext_alpn': {
                    'alpn_protocols': ["http/1.1"]
                }, '_selector': 0x0010
            },
            {
                'ext_sct': '', '_selector': 0x0012
            },
            {
                'ext_suppoerted_versions': {
                    'supported_versions': [0x0304, 0x0303, 0x0302, 0x0301]
                },
                '_selector': 0x002b
            },
            {
                'ext_key_share': {
                    'key_shares': [
                        {
                            'group': 0x001d, 'data': public_key
                        }
                    ]
                },
                '_selector': 0x0033
            }
        ]
    },
    '_selector': 0x01
}

client_hello_packet = {
    "tls_version": 0x0301,
    "tls_handshake": client_hello,
    "_selector": 22
}

In [116]:
client_hello_encoded = public_construct(client_hello_packet_spec, client_hello_packet)
client_hello_encoded

b'\x16\x03\x01\x00\xd4\x01\x00\x00\xd0\x03\x03\xabf\x18\xb2\xbf8\x98\x95\x0c\x1fH \x89w`1|4\x8c;\x9c\x82\x88\xa0^CmK\xa4c\x1d\x9d \x13\x02b\xb1\xedb=\x15\xda\xe1\x18\x90\\\xdeH\x93n\xdc\xf5x\xfaB-\x94lR\x85B=b4\xe6\x00\x02\x13\x01\x01\x00\x00\x85\x00\x00\x00\x13\x00\x11\x00\x00\x0ewww.google.com\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x04\x00\x02\x00\x1d\x00\x0b\x00\x02\x01\x00\x00\r\x00\x04\x00\x02\x04\x03\xff\x01\x00\x01\x00\x00\x10\x00\x0b\x00\t\x08http/1.1\x00\x12\x00\x00\x00+\x00\t\x08\x03\x04\x03\x03\x03\x02\x03\x01\x003\x00&\x00$\x00\x1d\x00 \xefi\xb5\xd8\x9e\xe5\xdc\r\xa0\xfd\x87G\x94\x16p\xf9\x1b\xb4B\x91V\xa8\x1c\xa8\x89\x9c\xda\xd9\x9c\xaf\x9a!'

In [117]:
transcript.update(client_hello_encoded[5:])

In [118]:
sock.send(client_hello_encoded)

In [119]:
def read_tls(sock):
    header = bytearray(sock.read_len(5))
    l = (header[3] << 8) + header[4]
    body = sock.read_len(l)
    return header + body

In [120]:
server_hello_encoded = read_tls(sock)

In [121]:
transcript.update(server_hello_encoded[5:])

In [122]:
server_hello_spec = """
8==2?24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16@extensions(
        ...(
            16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==16?16@ext_alpn(
                16@alpn_protocols(8())
            )
            :16==18?16@ext_sct(
                16@scts(
                    ...16@data()
                )
            )
            :16==43?16@ext_suppoerted_versions(
                16@supported_versions
            )
            :16==44?16@ext_cookie(
                16@cookie()
            )
            :16==51?16@ext_key_share(
                16@group
                -16@data()
            )
            :16==41?16@ext_pre_shared_key(
                16@label()
            )
            :16==16?16@ext_alpn(
                16@alpn(
                    8@data()
                )
            )
            :16@selector-16@raw_extension()
        )
    )
)
"""


server_hello_packet_spec = """
 8==22?16@tls_version-16@tls_handshake(%s)
:8==20?16@tls_version-16@change_cypher()
:8==21?16@tls_version-16@alert()
:8==23?16@tls_version-16@application_data()
"""%server_hello_spec


In [123]:
server_hello = public_match_exact(server_hello_packet_spec, server_hello_encoded)

In [124]:
server_hello

{'tls_version': 0x0303,
 '_selector': 0x16,
 'tls_handshake': {'server_hello': {'version': 0x0303,
   'random': 0x29ef579c8aab89748fb6c50b49235e409a65b21281f6f04579ca96163b19798d,
   'session_id': 0x130262b1ed623d15dae118905cde48936edcf578fa422d946c5285423d6234e6,
   'suite': 0x1301,
   'compression': 0x00,
   'extensions': [{'ext_key_share': {'group': 0x001d,
      'data': 0xc71704c192e0da854bfb18abe82134b4a6916c8ad5e55e23c9db9374f0199e35},
     '_selector': 0x0033},
    {'ext_suppoerted_versions': {'supported_versions': 0x0304},
     '_selector': 0x002b}]},
  '_selector': 0x02}}

In [125]:
server_public_key = None
for extension in server_hello['tls_handshake']['server_hello']['extensions']:
    if 'ext_key_share' in extension:
        print("found public key")
        server_public_key = extension['ext_key_share']['data'].bytes()
        break
    else:
        raise RuntimeError("unable to find server public key")


found public key


In [126]:
shared_key = private_key.exchange(X25519PublicKey.from_public_bytes(server_public_key))
print("calculated shared key", shared_key)

calculated shared key b'\x83=\x83\xe2\xdc\xfd\x9c\xd9\x98\xceb\x8eX"\xc3\xfaG9\x10\xb2\x89\xd8"\xafE}$\xcd\xcc\x1a\x9a/'


In [127]:
server_change_cipher_encoded = read_tls(sock)

In [128]:
server_change_cipher = public_match_exact(server_hello_packet_spec, server_change_cipher_encoded)
server_change_cipher

{'tls_version': 0x0303, '_selector': 0x14, 'change_cypher': 0x01}

In [129]:
sock.send(server_change_cipher_encoded)

In [130]:
def get_info(label, length, context=b''):
    return b'\x00' + bytearray((length, len(b'tls13 '+label),)) + b'tls13 ' + label + bytearray((len(context),)) + context

def extract(secret, salt):
    return HKDF(hashes.SHA256(), 32, salt, b'')._extract(secret)    
    
def derive(secret, label, prev=None):
    context = get_hash(prev)
    return expand(secret, label, 32, context)
    
def expand(secret, label, l, context=b''):
    return HKDFExpand(hashes.SHA256(), l, get_info(label, l, context)).derive(secret)

def get_hash(prev):
    if prev is None:
        prev = hashes.Hash(hashes.SHA256())
    c = prev.copy()
    return c.finalize()

def finished_hash(secret, prev=None):
    finished_key = expand(secret, b'finished', 32)
    context = get_hash(prev)
    h = HMAC(finished_key, hashes.SHA256())
    h.update(context)
    return h.finalize()
    

In [131]:
early_secret = extract(b'\0'*32, None)

extractor = derive(early_secret, b"derived")

handshake_secret = extract(shared_key, extractor)

In [132]:
server_handshake_secret = derive(handshake_secret, b"s hs traffic", transcript)

server_handshake_key = expand(server_handshake_secret, b'key', 16)
server_handshake_nonce = (expand(server_handshake_secret, b'iv', 12), 0)

In [133]:
client_handshake_secret = derive(handshake_secret, b"c hs traffic", transcript)

client_handshake_key = expand(client_handshake_secret, b'key', 16)
client_handshake_nonce = (expand(client_handshake_secret, b'iv', 12), 0)

In [134]:
server_cert_encrypted_encoded = read_tls(sock)

In [135]:
server_cert_encrypted = public_match_exact(server_hello_packet_spec, server_cert_encrypted_encoded)
server_cert_encrypted

{'tls_version': 0x0303,
 '_selector': 0x17,
 'application_data': 0x9feb2477fafe700bf3100a843558c74374f9eb4a06f787795a593708ccf0249383a01d278fdf6a3242ecbd467a1159fccde45de8b1e2e234502dcc41e1492b1173acc57b40bd7bd8d524b56314a7b719768c64a60d882c1c2b3e3ee5fbc86d961a7a869708b036c90309fe0b24a68abf89d8af51d76578e29258c6566460c07d99a823bb06273fa8f1385410409ae457b1f906b379da6bfb856178d94a2b52590910ecf2f528bca50d89f59a3fc32f319d3f8e1beeb5caa9cfcc58c38609a4d75ff01c9e972b2abd7a867eb505002c298153d2eb448a165bff7c82e5864d738ed6d61ebb45180a486fa1790801df1609e0e0298b10e2ef090ef83715c7ab98d3b7f3304e0ae7cf1d832f7b19fdf85188ef93485c0e089f61fd12238fc90e610b5df6114b377bfe92d42fecd1cb6bcaaa12a08fb8f07f8ec700fdec80aaf54e2a3b88691a79ec475c89b4bdcebc0ab8c00afc3482c258f4cc253527ab50808e72738f4390c014a1827ab617565d82ac108eb95df1a69b92f2584d29a7f60c33b90a4c9e522a994a4b3e8d796cd2c5aeca4fb0c638a0cd175040a96187fa94976febb5850fbde4f71efff0413f5ece64ebe3d3956dc002768f239cf89c7eac296a666d36ec318b3f856339102d2c576ac272e5e

In [136]:
def update_nonce(nonce):
    return (nonce[0], nonce[1]+1)

def get_nonce(nonce):
    return bytes(a ^ b for a, b in zip(nonce[0], nonce[1].to_bytes(len(nonce[0]), 'big')))

In [137]:
def decrypt_packet(packet, key, nonce):
    packet = bytes(packet)
    cipher = AESGCM(key)
    all_server_cert_packet = cipher.decrypt(get_nonce(nonce), packet[5:], packet[:5])
    server_cert_packet, server_cert_packet_suffix = all_server_cert_packet[:-1], all_server_cert_packet[-1]
    return server_cert_packet, server_cert_packet_suffix, update_nonce(nonce)

def encrypt_packet(packet, suffix, key, nonce):
    app_header_spec = "8@packet_type-16@tls_version-16@application_data_length"

    raw_packet = packet + suffix

    header_encoded = public_construct(app_header_spec, {
        "packet_type": 23,
        "tls_version": 0x0303,
        "application_data_length": len(raw_packet) + 16,
    })

    
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(get_nonce(nonce), raw_packet, header_encoded)
    packet = header_encoded + encrypted_app_data
    return packet, update_nonce(nonce)

In [138]:
server_cert_packet, server_cert_packet_suffix, server_handshake_nonce = decrypt_packet(server_cert_encrypted_encoded, server_handshake_key, server_handshake_nonce)
print(server_cert_packet_suffix)
server_cert_packet

22


b'\x08\x00\x00\x11\x00\x0f\x00\x10\x00\x0b\x00\t\x08http/1.1\x0b\x00\x0f\x9d\x00\x00\x0f\x99\x00\x04\x8a0\x82\x04\x860\x82\x03n\xa0\x03\x02\x01\x02\x02\x10$Wu\xfe\xc4S\xb4\xb0\n\x00\x00\x00\x00\xfff\x0b0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x0b\x05\x000F1\x0b0\t\x06\x03U\x04\x06\x13\x02US1"0 \x06\x03U\x04\n\x13\x19Google Trust Services LLC1\x130\x11\x06\x03U\x04\x03\x13\nGTS CA 1C30\x1e\x17\r210913040713Z\x17\r211120040712Z0\x191\x170\x15\x06\x03U\x04\x03\x13\x0ewww.google.com0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04O\xb7\x02\xc1\xaaV\xb5\xa2a`\xc7\x04\x9fl)65\xd9K\xc9\x9b3e\xb0\xe4qUY\xa6\n\x98X\xcb\xebq\xde\xcd\x06\xd5\xdf\xbb\x8dcX\xca.Z\xca\xd2\x8b\xd7\x0e\x9c\xb2\xe3\xcbdK@\xb2\x90\'^3\xa3\x82\x02f0\x82\x02b0\x0e\x06\x03U\x1d\x0f\x01\x01\xff\x04\x04\x03\x02\x07\x800\x13\x06\x03U\x1d%\x04\x0c0\n\x06\x08+\x06\x01\x05\x05\x07\x03\x010\x0c\x06\x03U\x1d\x13\x01\x01\xff\x04\x020\x000\x1d\x06\x03U\x1d\x0e\x04\x16\x04\x14bz\x0b\xc0\x8d\x9f\x13\x9c\x9e\x97\x

In [139]:
multiple_packets = server_cert_packet

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
encrypted_extensions_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificates_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
certificate_verify_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

packet_len = (multiple_packets[1] << 16) + (multiple_packets[2] << 8) + multiple_packets[3]
finished_message_packet, multiple_packets = multiple_packets[:4+packet_len], multiple_packets[4+packet_len:]

In [140]:
handshake_app_data_spec = """
    8==8?24@encrypted_extenstions(
        16@data(
            16@alpn
            -16@data(
                16@data(
                    8@data()
                )
            )
        )
    )
    :8==11?24@certificates_data(
        8@certificate_request_context-24@certificates_data(
            ...(
                24@certificate()
                -16@extensions(
                    16==5?16@status_request(
                        8@status_type_ocsp
                        -24@ocsp_staple(
                        )
                    )
                    :16==18?16@sct(
                        16@data(
                            ...16@data()
                        )
                    )
                )
            )
        )
    )
    :8==15?24@certificate_verify(
        16@signature_algorithm-16@signature()
    )
    :8==20?24@finished_verify_data()
    :8==22?24@certificate_status(
        8@status_type
        -24@response()
    )
    :8@selector-24@data()
"""

In [141]:
encrypted_extensions = public_match_exact(handshake_app_data_spec, encrypted_extensions_packet)
encrypted_extensions

{'encrypted_extenstions': {'data': {'alpn': 0x0010,
   'data': {'data': {'data': "http/1.1"}}}},
 '_selector': 0x08}

In [142]:
server_cert = public_match_exact(handshake_app_data_spec, certificates_packet)
server_cert

{'certificates_data': {'certificate_request_context': 0x00,
  'certificates_data': [{'certificate': 0x308204863082036ea0030201020210245775fec453b4b00a00000000ff660b300d06092a864886f70d01010b05003046310b300906035504061302555331223020060355040a1319476f6f676c65205472757374205365727669636573204c4c43311330110603550403130a47545320434120314333301e170d3231303931333034303731335a170d3231313132303034303731325a3019311730150603550403130e7777772e676f6f676c652e636f6d3059301306072a8648ce3d020106082a8648ce3d030107034200044fb702c1aa56b5a26160c7049f6c293635d94bc99b3365b0e4715559a60a9858cbeb71decd06d5dfbb8d6358ca2e5acad28bd70e9cb2e3cb644b40b290275e33a382026630820262300e0603551d0f0101ff04040302078030130603551d25040c300a06082b06010505070301300c0603551d130101ff04023000301d0603551d0e04160414627a0bc08d9f139c9e97880d93c1de4e10a991fc301f0603551d230418301680148a747faf85cdee95cd3d9cd0e24614f371351d27306a06082b06010505070101045e305c302706082b06010505073001861b687474703a2f2f6f6373702e706b692e676f6f672f67747331633330

In [143]:
import OpenSSL.crypto

first_cert = None
for cert in server_cert['certificates_data']['certificates_data']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].bytes())
    if first_cert is None:
        first_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            24:57:75:fe:c4:53:b4:b0:0a:00:00:00:00:ff:66:0b
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Google Trust Services LLC, CN=GTS CA 1C3
        Validity
            Not Before: Sep 13 04:07:13 2021 GMT
            Not After : Nov 20 04:07:12 2021 GMT
        Subject: CN=www.google.com
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:4f:b7:02:c1:aa:56:b5:a2:61:60:c7:04:9f:6c:
                    29:36:35:d9:4b:c9:9b:33:65:b0:e4:71:55:59:a6:
                    0a:98:58:cb:eb:71:de:cd:06:d5:df:bb:8d:63:58:
                    ca:2e:5a:ca:d2:8b:d7:0e:9c:b2:e3:cb:64:4b:40:
                    b2:90:27:5e:33
                ASN1 OID: prime256v1
                NIST CURVE: P-256
        X509v3 extensions:
            X509v3 Key Usage: critical
                Digital 

In [144]:
cert_pub = first_cert.get_pubkey().to_cryptography_key()

In [145]:
transcript.update(encrypted_extensions_packet)
transcript.update(certificates_packet)

In [146]:
cert_verify_message = public_match_exact(handshake_app_data_spec, certificate_verify_packet)
cert_verify_message

{'certificate_verify': {'signature_algorithm': 0x0403,
  'signature': 0x304502203016fa8d2d1bc6b97f13325e61ce3326bd81c52c3a80e24fd39aaa8702814a7d022100f7d5550a2603da7e7befccc5672e8d52d2799d901047ca462fbf95366252cda4},
 '_selector': 0x0f}

In [147]:
sign_data = b'\x20' * 64 + \
    b'TLS 1.3, server CertificateVerify\x00' + \
    get_hash(transcript)

signature = cert_verify_message['certificate_verify']['signature'].bytes()

cert_pub.verify(signature, sign_data, ec.ECDSA(hashes.SHA256()))
print("certificate verify succeeded")

certificate verify succeeded


In [148]:
transcript.update(certificate_verify_packet)

In [149]:
finished_message = public_match_exact(handshake_app_data_spec, finished_message_packet)
finished_message

{'finished_verify_data': 0xa1ea076b7e7ea3f386aedb9e04f548441a70d0e464f12dcaa3a8f3d68e907929,
 '_selector': 0x14}

In [150]:
if finished_hash(server_handshake_secret, transcript) == finished_message['finished_verify_data'].bytes():
    print("matching finished hash")
else:
    raise RuntimeError("not matching finished hash")

matching finished hash


In [151]:
transcript.update(finished_message_packet)

In [152]:
client_finished_hash = finished_hash(client_handshake_secret, transcript)
client_verify_unencrypted = public_construct(handshake_app_data_spec, {'finished_verify_data': client_finished_hash, '_selector': 0x14})

In [153]:
client_verify_packet, client_handshake_nonce = encrypt_packet(
    client_verify_unencrypted, b'\x16', client_handshake_key, client_handshake_nonce
)

In [154]:
sock.send(client_verify_packet)

In [155]:
master_secret = extract(b'\0' * 32, derive(handshake_secret, b"derived"))
server_app_secret = derive(master_secret, b"s ap traffic", transcript)
server_app_key = expand(server_app_secret, b'key', 16)
server_app_nonce = (expand(server_app_secret, b'iv', 12), 0)
client_app_secret = derive(master_secret, b"c ap traffic", transcript)
client_app_key = expand(client_app_secret, b'key', 16)
client_app_nonce = (expand(client_app_secret, b'iv', 12), 0)

In [156]:
transcript.update(client_verify_packet)

In [157]:
http_req = b"""GET / HTTP/1.1
Host: www.google.com

"""
packet, client_app_nonce = encrypt_packet(
    http_req, b'\x17', client_app_key, client_app_nonce
)

In [158]:
sock.send(packet)

In [159]:
http_resp = read_tls(sock)

In [160]:
data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
(data, packet_type, server_app_nonce)

(b'HTTP/1.1 200 OK\r\nDate: Tue, 05 Oct 2021 12:46:31 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: CONSENT=PENDING+061; expires=Fri, 01-Jan-2038 00:00:00 GMT; path=/; domain=.google.com; Secure\r\nAlt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"\r\nAccept-Ranges: none\r\nVary: Accept-Encoding\r\nTransfer-Encoding: chunked\r\n\r\n5116\r\n<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="nl"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="CmY+0qKDK4xh+D

In [161]:
class HttpsReader(ReaderWrapper):
    def _read(self):
        global server_app_nonce
        http_resp = read_tls(sock)
        data, packet_type, server_app_nonce = decrypt_packet(http_resp, server_app_key, server_app_nonce)
        print("got body", data[:100])
        return data

In [163]:
http_reader = HttpsReader(initial_value=data)

In [164]:
http_reader.read_line()

b'HTTP/1.1 200 OK\r\n'

In [165]:
content_length = None
chunked = False

while True:
    header = http_reader.read_line()
    print(header)
    if header == b'\r\n':
        break
    if header.lower().startswith(b'content-length:'):
        content_length = int(i.split(':').strip())
        print('got content length', content_length)
    elif header.lower().startswith(b'transfer-encoding: chunked'):
        chunked = True
        print('got chunked')


b'Date: Tue, 05 Oct 2021 12:46:31 GMT\r\n'
b'Expires: -1\r\n'
b'Cache-Control: private, max-age=0\r\n'
b'Content-Type: text/html; charset=ISO-8859-1\r\n'
b'P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\n'
b'Server: gws\r\n'
b'X-XSS-Protection: 0\r\n'
b'X-Frame-Options: SAMEORIGIN\r\n'
b'Set-Cookie: CONSENT=PENDING+061; expires=Fri, 01-Jan-2038 00:00:00 GMT; path=/; domain=.google.com; Secure\r\n'
b'Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"\r\n'
b'Accept-Ranges: none\r\n'
b'Vary: Accept-Encoding\r\n'
b'Transfer-Encoding: chunked\r\n'
got chunked
b'\r\n'


In [166]:
body = b''

In [167]:
while chunked:
    print("reading chunks")
    length = http_reader.read_line()
    print("length line", length)
    l = int(length[:-2], 16)

    ch = http_reader.read_len(l)
    print("got chunk", len(ch), ch[:100])
    body += ch
    http_reader.read_line()
    if l == 0:
        break

reading chunks
length line b'5116\r\n'
got body b',1528,2304,1236,5227,5260,2014,13611,4764,2658,4242,2459,656,30,13628,2305,638,1494,16786,2521,3285,'
got body b'+l(d),-1===b.search("&lei=")&&(d=m(d))&&(e+="&lei="+d));d="";!c&&f._cshid&&-1===b.search("&cshid=")&'
got body b'nt;a=a.parentElement)if("A"===a.tagName){a="1"===a.getAttribute("data-nohref");break a}a=!1}a&&b.pre'
got body b':0}#gbg .gbm{right:0}.gbxms{background-color:#ccc;display:block;position:absolute;z-index:1;top:-1px'
got body b':inline}.gbzt .gbts{display:inline;zoom:1}.gbto .gbts{background:#fff;border-color:#bebebe;color:#36'
got body b'on:0 0}#gbmpi,#gbmpid{border:none;display:inline-block;height:48px;width:48px}#gbmpiw{display:inline'
got body b'er;outline:0 solid black;text-decoration:none !important}.gbm0l,.gbm0l:visited{color:#000 !important'
got body b'px 20px}#gbpm{border:0;*border-collapse:collapse;border-spacing:0;margin:0;white-space:normal}#gbpm '
got body b'set 0 0 0 1px rgba(255, 255, 255, 0.5);box-sha

In [168]:
from IPython.core.display import display, HTML
display(HTML(body.decode("latin1")))

,,Geavanceerd zoeken


In [169]:
sock.close()